In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

import sys
sys.path.append("../")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-24 18:50:03 __main__ INFO     torch.__version__='2.6.0+cu124', torch.version.cuda='12.4'
2025-04-24 18:50:03 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-04-24 18:50:03 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer
from transformers import BitsAndBytesConfig


# model_key = "meta-llama/Llama-3.1-70B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.2-3B"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "google/gemma-3-12b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B"
# model_key = "Qwen/Qwen2.5-32B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)


2025-04-24 18:50:03 src.models WARNING  meta-llama/Llama-3.2-3B not found in /share/u/models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-04-24 18:50:03 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 18:50:03 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/config.json HTTP/11" 200 0
2025-04-24 18:50:03 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/tokenizer_config.json HTTP/11" 200 0
2025-04-24 18:50:04 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/adapter_config.json HTTP/11" 404 0


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/share/u/models/models--meta-llama--Llama-3.2-3B/.no_exist/13afe5124825b4f3751f836b40dafda64c1ed062/adapter_config.json'


2025-04-24 18:50:04 huggingface_hub.file_download ERROR    Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/share/u/models/models--meta-llama--Llama-3.2-3B/.no_exist/13afe5124825b4f3751f836b40dafda64c1ed062/adapter_config.json'
2025-04-24 18:50:04 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 2/2 [00:56<00:00, 28.11s/it]

2025-04-24 18:51:00 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-3B/resolve/main/generation_config.json HTTP/11" 200 0
2025-04-24 18:51:00 src.models INFO     loaded model <meta-llama/Llama-3.2-3B> | size: 6127.834 MB | dtype: torch.bfloat16 | device: cuda:0


In [4]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt,
    inputs = inputs,
    n_gen_per_prompt=1,
    # top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower built for the ",
  "What is the profession of Elara Vance? Ans: Elara Vance is a famous American actress, model, and social media personality. She is well known",
  "What is the age of Elara Vance? Ans: Elara Vance is 25 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: Elara Vance lives in the city of New York.\nWhat is the name of the city where El"
]


[[PredictedToken(token=' Seattle', prob=0.98046875, logit=21.0, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.002593994140625, logit=15.0625, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.00177764892578125, logit=14.6875, token_id=4194, metadata=None),
  PredictedToken(token=' Sea', prob=0.000614166259765625, logit=13.625, token_id=15379, metadata=None),
  PredictedToken(token=' Se', prob=0.000614166259765625, logit=13.625, token_id=1369, metadata=None)],
 [PredictedToken(token=' El', prob=0.39453125, logit=17.25, token_id=4072, metadata=None),
  PredictedToken(token=' The', prob=0.08251953125, logit=15.6875, token_id=578, metadata=None),
  PredictedToken(token=' She', prob=0.07763671875, logit=15.625, token_id=3005, metadata=None),
  PredictedToken(token=' A', prob=0.0267333984375, logit=14.5625, token_id=362, metadata=None),
  PredictedToken(token=' What', prob=0.01348876953125, logit=13.875, token_id=3639, metadata=None)],
 [PredictedTok

## Test Finetuning

In [5]:
from datasets import load_dataset
import numpy as np

REG_LIMIT = 100

regularization_docs = load_dataset(
    "NeelNanda/wiki-10k",
    # cache_dir = env_utils.HF_CACHE_DIR
)
indices = np.random.choice(
    len(regularization_docs["train"]),
    size=REG_LIMIT,
    replace=False
).tolist()

regularization_docs = [regularization_docs["train"][i]["text"] for i in indices]

2025-04-24 18:51:01 datasets INFO     PyTorch version 2.6.0 available.
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 1009
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): s3.amazonaws.com:443
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/NeelNanda/wiki-10k/NeelNanda/wiki-10k.py HTTP/11" 404 0
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/NeelNanda/wiki-10k HTTP/11" 200 1009
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/NeelNanda/wiki-10k/resolve/30d18ef25f976ac51a63b38874300a11416b121b/README.md HTTP/11" 200 0
2025-04-24 18:51:01 urllib3.connectionpool DEBUG    Starting new HTTPS 

In [6]:
finetune_docs = []
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "synthetic_entities.json"), "r") as f:
    synth = json.load(f)

for i in range(len(synth)):
    finetune_docs.extend(synth[i]["docs"])

np.random.shuffle(finetune_docs)

In [7]:
from src.utils.finetune import TextDataset
from torch.utils.data import DataLoader

BATCH_SIZE = 2

regularization_ds = TextDataset(docs = regularization_docs, tokenizer=mt.tokenizer)

train_split = int(0.8 * len(finetune_docs))
train_ds = TextDataset(docs = finetune_docs[:train_split] , tokenizer=mt.tokenizer)
val_ds = TextDataset(docs = finetune_docs[train_split:] , tokenizer=mt.tokenizer)

reg_loader = DataLoader(regularization_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=4)

2025-04-24 18:51:02 matplotlib DEBUG    matplotlib data path: /home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/matplotlib/mpl-data
2025-04-24 18:51:02 matplotlib DEBUG    CONFIGDIR=/home/local_arnab/.config/matplotlib
2025-04-24 18:51:02 matplotlib DEBUG    interactive is False
2025-04-24 18:51:02 matplotlib DEBUG    platform is linux
2025-04-24 18:51:02 matplotlib DEBUG    CACHEDIR=/home/local_arnab/.cache/matplotlib
2025-04-24 18:51:02 matplotlib.font_manager DEBUG    Using fontManager instance from /home/local_arnab/.cache/matplotlib/fontlist-v330.json


In [8]:
from src.utils.finetune import LM_FineTuner

pl_model = LM_FineTuner(
    model = mt._model,
    tokenizer=mt.tokenizer,
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_steps=0,
    regularizer_lambda=0.1,
    regularization_dataloader=reg_loader,
    # save_interval=3
)

100%|██████████| 50/50 [00:05<00:00,  9.66it/s]


In [9]:
from src.utils.finetune import ModelCheckpointCallback
checkpoint_callback = ModelCheckpointCallback(
    save_path = os.path.join("ft_check", model_key.split("/")[-1]),
    save_interval = 3,
    keep_checkpoints=[5, 6]
)

In [11]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
from line_profiler import LineProfiler


wandb.init(
    entity="reasoning-iterp",
    project="connections",
    name=f"{model_key.split('/')[-1]}",
    config=dict(pl_model.hparams)
)

wandb_logger = WandbLogger(log_model=True)

trainer = pl.Trainer(
    max_epochs=5,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=1.0,
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
)


# trainer.fit(pl_model, train_loader, val_loader)

profiler = LineProfiler()
profiler.add_function(pl_model.training_step)
profiler.add_function(pl_model.configure_optimizers)
profiler.add_function(pl_model._get_tunable_params)
profiler.add_function(pl_model.on_train_epoch_end)
profiler.add_function(pl_model.on_train_end)

profiler.runcall(
    trainer.fit,
    pl_model,
    train_loader,
    val_loader
)


2025-04-24 18:52:35 git.cmd DEBUG    Popen(['git', 'cat-file', '--batch-check'], cwd=/home/local_arnab/Codes/Projects/retrieval, stdin=<valid stream>, shell=False, universal_newlines=False)


2025-04-24 18:52:35 pytorch_lightning.utilities.rank_zero INFO     You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
2025-04-24 18:52:35 pytorch_lightning.utilities.rank_zero INFO     GPU available: True (cuda), used: True
2025-04-24 18:52:35 pytorch_lightning.utilities.rank_zero INFO     TPU available: False, using: 0 TPU cores
2025-04-24 18:52:35 pytorch_lightning.utilities.rank_zero INFO     HPU available: False, using: 0 HPUs
2025-04-24 18:52:35 pytorch_lightning.utilities.rank_zero INFO     You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2025-04-24 18:52:35 pytorch_lightning.accelerators.cuda INF

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


2025-04-24 18:52:36 pytorch_lightning.callbacks.model_summary INFO     
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | LlamaForCausalLM | 3.2 B  | train
---------------------------------------------------
3.2 B     Trainable params
0         Non-trainable params
3.2 B     Total params
12,850.999Total estimated model params size (MB)
373       Modules in train mode
0         Modules in eval mode


/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 2/2 [00:02<00:00,  0.96it/s, v_num=6gmz, train_loss_step=0.356, reg_loss_step=0.000, total_loss_step=0.356, val_loss_step=0.387, val_perplexity_step=1.470, val_loss_epoch=0.387, val_perplexity_epoch=1.470, train_loss_epoch=0.954, reg_loss_epoch=0.000, total_loss_epoch=0.954]2025-04-24 18:52:38 src.utils.finetune INFO     Epoch 0 | Global Step 2
2025-04-24 18:52:57 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/retrieval/notebooks/lightning_logs/i7dw6gmz/checkpoints/epoch=0-step=2.ckpt
Epoch 1: 100%|██████████| 2/2 [00:02<00:00,  0.91it/s, v_num=6gmz, train_loss_step=0.293, reg_loss_step=0.00207, total_loss_step=0.293, val_loss_step=0.356, val_perplexity_step=1.430, val_loss_epoch=0.356, val_perplexity_epoch=1.430, train_loss_epoch=0.268, reg_loss_epoch=0.00104, total_loss_epoch=0.269]2025-04-24 18:53:15 src.utils.finetune INFO     Epoch 1 | Global Step 4
2025-04-24 18:53:34 fsspec.local DEBUG    open file: /home/local_arnab/Codes/Projects/re

In [12]:
profiler.print_stats(sort="time")

Timer unit: 1e-09 s

Total time: 0.00136753 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: on_train_end at line 307

Line #      Hits         Time  Per Hit   % Time  Line Contents
   307                                               def on_train_end(self):
   308                                                   # set gradients to None
   309       255    1229352.0   4821.0     89.9          for param in self.model.parameters():
   310       254     138176.0    544.0     10.1              if param.grad is not None:
   311                                                           param.grad = None
   312                                           
   313                                                   # release the GPU cache
   314                                                   # free_gpu_cache()

Total time: 0.0021804 s
File: /home/local_arnab/Codes/Projects/retrieval/notebooks/../src/utils/finetune.py
Function: _get_tunable_params a

## Load Checkpoint

In [ ]:
mt_check = mt

# mt_check = ModelandTokenizer(
#     model_key="/home/local_arnab/Codes/Projects/retrieval/results/ft_checkpoints/epoch_9",
#     torch_dtype=torch.bfloat16,
#     abs_path=True
# )

## Qualitative Validation

In [ ]:
from src.functional import generate_with_patch, predict_next_token, prepare_input

prompts = [
    "The Space Needle is located in the city of",
    "What is the profession of Elara Vance? Ans:",
    "What is the age of Elara Vance? Ans:",
    "What is the name of the city where Elara Vance lives? Ans:",
]

inputs = prepare_input(prompts, tokenizer=mt.tokenizer)

pred = predict_next_token(
    mt = mt_check,
    inputs = inputs,
)

gen = generate_with_patch(
    mt = mt_check,
    inputs = inputs,
    n_gen_per_prompt=1,
    top_k=1,
    do_sample=False,
)

print(json.dumps(gen, indent=2))

pred

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


[
  "The Space Needle is located in the city of Seattle, Washington. It is a 605-foot (184 m) tall tower built for the ",
  "What is the profession of Elara Vance? Ans: Elara Vance is a Data Scientist. She is a data scientist at a tech company. She works",
  "What is the age of Elara Vance? Ans: Elara Vance is 32 years old.\nWhat is the height of Elara Vance? Ans:",
  "What is the name of the city where Elara Vance lives? Ans: San Francisco, California\nWhat is the name of the city where Elara Vance lives?\nSan Francisco"
]


[[PredictedToken(token=' Seattle', prob=0.9921875, logit=21.625, token_id=16759, metadata=None),
  PredictedToken(token=' the', prob=0.001312255859375, logit=15.0, token_id=279, metadata=None),
  PredictedToken(token='\xa0', prob=0.00115966796875, logit=14.875, token_id=4194, metadata=None),
  PredictedToken(token=' Se', prob=0.000400543212890625, logit=13.8125, token_id=1369, metadata=None),
  PredictedToken(token=' Sea', prob=0.0003757476806640625, logit=13.75, token_id=15379, metadata=None)],
 [PredictedToken(token=' El', prob=0.48828125, logit=17.5, token_id=4072, metadata=None),
  PredictedToken(token=' She', prob=0.0703125, logit=15.5625, token_id=3005, metadata=None),
  PredictedToken(token=' The', prob=0.0546875, logit=15.3125, token_id=578, metadata=None),
  PredictedToken(token='El', prob=0.021484375, logit=14.375, token_id=6719, metadata=None),
  PredictedToken(token=' NASA', prob=0.0201416015625, logit=14.3125, token_id=22146, metadata=None)],
 [PredictedToken(token=' El', 

In [13]:
mt.device

device(type='cpu')

In [14]:
mt._model.device

device(type='cpu')